In [7]:
import math
import random

class RNN:
    def __init__(self, vocab_size, hidden_size):
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.W_xh = self.init_matrix(hidden_size, vocab_size)
        self.W_hh = self.init_matrix(hidden_size, hidden_size)
        self.W_hy = self.init_matrix(vocab_size, hidden_size)
        self.b_h = [0.0] * hidden_size
        self.b_y = [0.0] * vocab_size

    def init_matrix(self, rows, cols):
        return [[random.uniform(-0.1, 0.1) for _ in range(cols)] for _ in range(rows)]
    def tanh(self, v):
        return [math.tanh(x) for x in v]
    def tanh_derivative(self, v):
        return [1 - math.tanh(x)**2 for x in v]
    def softmax(self, v):
        exps = [math.exp(x) for x in v]
        total = sum(exps)
        return [x / total for x in exps]

    def mat_vec_mul(self, mat, vec):
        return [sum(m * v for m, v in zip(row, vec)) for row in mat]
    def vec_add(self, v1, v2):
        return [a + b for a, b in zip(v1, v2)]
    def forward(self, inputs):
        h = [0.0] * self.hidden_size
        self.h_list = []

        for x in inputs:
            h = self.vec_add(
                self.vec_add(self.mat_vec_mul(self.W_xh, x), self.mat_vec_mul(self.W_hh, h)),
                self.b_h
            )
            h = self.tanh(h)
            self.h_list.append(h)

        logits = self.vec_add(self.mat_vec_mul(self.W_hy, h), self.b_y)
        return self.softmax(logits)

    def backward(self, inputs, target_index, y_pred):
        dW_xh = [[0.0] * self.vocab_size for _ in range(self.hidden_size)]
        dW_hh = [[0.0] * self.hidden_size for _ in range(self.hidden_size)]
        dW_hy = [[0.0] * self.hidden_size for _ in range(self.vocab_size)]
        db_h = [0.0] * self.hidden_size
        db_y = [0.0] * self.vocab_size

        dy = y_pred[:]
        dy[target_index] -= 1
        h_last = self.h_list[-1]
        for i in range(self.vocab_size):
            for j in range(self.hidden_size):
                dW_hy[i][j] += dy[i] * h_last[j]
            db_y[i] += dy[i]

# Backpropagation
        dh = [sum(self.W_hy[i][j] * dy[i] for i in range(self.vocab_size)) for j in range(self.hidden_size)]

        for t in reversed(range(len(inputs))):
            dh_raw = [d * d_tanh for d, d_tanh in zip(dh, self.tanh_derivative(self.h_list[t]))]
            x_t = inputs[t]
            h_prev = self.h_list[t - 1] if t > 0 else [0.0] * self.hidden_size

            for i in range(self.hidden_size):
                for j in range(self.vocab_size):
                    dW_xh[i][j] += dh_raw[i] * x_t[j]
                for j in range(self.hidden_size):
                    dW_hh[i][j] += dh_raw[i] * h_prev[j]
                db_h[i] += dh_raw[i]

            dh = [sum(self.W_hh[i][j] * dh_raw[i] for i in range(self.hidden_size)) for j in range(self.hidden_size)]

        return dW_xh, dW_hh, db_h, dW_hy, db_y

    def update(self, grads, lr):
        dW_xh, dW_hh, db_h, dW_hy, db_y = grads
        for i in range(self.hidden_size):
            for j in range(self.vocab_size):
                self.W_xh[i][j] -= lr * dW_xh[i][j]
            for j in range(self.hidden_size):
                self.W_hh[i][j] -= lr * dW_hh[i][j]
            self.b_h[i] -= lr * db_h[i]
        for i in range(self.vocab_size):
            for j in range(self.hidden_size):
                self.W_hy[i][j] -= lr * dW_hy[i][j]
            self.b_y[i] -= lr * db_y[i]

    def cross_entropy(self, pred, target):
        return -math.log(pred[target] + 1e-10)

    def train(self, inputs, target_index, lr=0.1, epochs=100):
        for epoch in range(epochs):
            pred = self.forward(inputs)
            loss = self.cross_entropy(pred, target_index)
            grads = self.backward(inputs, target_index, pred)
            self.update(grads, lr)

            if epoch % 10 == 0 or epoch == epochs - 1:
                acc = 100 if pred.index(max(pred)) == target_index else 0
                print(f"Epoch {epoch+1}/{epochs} Loss: {loss:.4f} Accuracy: {acc:.2f}%")

if __name__ == "__main__":
    words = ["we", "are", "best", "friends"]
    word_to_idx = {w: i for i, w in enumerate(words)}
    idx_to_word = {i: w for w, i in word_to_idx.items()}
    inputs = ["we", "are", "best"]
    target = "friends"
    input_vectors = [[1 if i == word_to_idx[word] else 0 for i in range(4)] for word in inputs]
    target_index = word_to_idx[target]
    rnn = RNN(vocab_size=4, hidden_size=4)
    rnn.train(input_vectors, target_index, epochs=100)
    prediction = rnn.forward(input_vectors)
    predicted_word = idx_to_word[prediction.index(max(prediction))]
    print(f"\nPredicted: '{predicted_word}' | Target: '{target}'")


Epoch 1/100 Loss: 1.3965 Accuracy: 0.00%
Epoch 11/100 Loss: 0.7697 Accuracy: 100.00%
Epoch 21/100 Loss: 0.4199 Accuracy: 100.00%
Epoch 31/100 Loss: 0.2362 Accuracy: 100.00%
Epoch 41/100 Loss: 0.1454 Accuracy: 100.00%
Epoch 51/100 Loss: 0.0984 Accuracy: 100.00%
Epoch 61/100 Loss: 0.0717 Accuracy: 100.00%
Epoch 71/100 Loss: 0.0552 Accuracy: 100.00%
Epoch 81/100 Loss: 0.0443 Accuracy: 100.00%
Epoch 91/100 Loss: 0.0366 Accuracy: 100.00%
Epoch 100/100 Loss: 0.0315 Accuracy: 100.00%

Predicted: 'friends' | Target: 'friends'
